In [7]:
# === Imports and Constants ===
import numpy as np
from scipy.spatial.distance import cdist
from collections import defaultdict, deque
import pandas as pd

# Environment constants
ENV_SIZE = 2.0                      # Size of the 2D environment
NUM_SERVERS = 5
NUM_USERS = 10
SERVER_SPEEDS = np.array([10,8, 4, 2, 1])
SLOT_DURATION = 5                # Time slot duration in seconds

In [2]:
# === Utility Functions ===

def generate_positions():
    """Randomly generate server and user positions."""
    servers_pos = np.random.uniform(0, ENV_SIZE, size=(NUM_SERVERS, 2))
    users_pos = np.random.uniform(0, ENV_SIZE, size=(NUM_USERS, 2))
    return servers_pos, users_pos

def compute_preferences(users_pos, servers_pos, server_speeds):
    """
    Compute preference rankings:
    - Users prefer servers by inverse of (5/speed + distance/5)
    - Servers prefer users by inverse of distance
    """
    dist_matrix = cdist(users_pos, servers_pos)
    user_pref_scores = 1.0 / ((5.0 / server_speeds) + dist_matrix / 5.0)
    server_pref_scores = 1.0 / dist_matrix.T
    user_preferences = np.argsort(-user_pref_scores, axis=1)
    server_preferences = np.argsort(-server_pref_scores, axis=1)
    return dist_matrix, user_preferences, server_preferences

def generate_workloads(num_users, distribution='exponential'):
    """Generate random workloads for users' tasks."""
    if distribution == 'exponential':
        return np.random.exponential(scale=5.0, size=num_users)
    elif distribution == 'uniform':
        return np.random.uniform(0, 10, size=num_users)
    else:
        raise ValueError("Invalid distribution")

In [3]:
# === Stable Matching ===

def many_to_one_stable_matching(user_prefs, server_prefs, server_capacity):
    """Gale-Shapley stable matching with capacity constraint."""
    num_users, num_servers = user_prefs.shape
    free_users = deque(range(num_users))                             # all users start free
    proposals = np.zeros((num_users, num_servers), dtype=bool)       # keep track of proposals
    server_matches = defaultdict(list)                               # who is matched to which server

    # Server rank map for quick preference lookup
    server_rank = np.full((num_servers, num_users), np.inf)          # ranks of users for servers
    for s in range(num_servers):
        for rank, u in enumerate(server_prefs[s]):
            server_rank[s, u] = rank

    # Proposal loop
    while free_users:
        u = free_users.popleft()
        for s in user_prefs[u]:
            if not proposals[u, s]:
                proposals[u, s] = True
                server_matches[s].append(u)
                break

        # Enforce server capacity
        for s in range(num_servers):
            current_users = server_matches[s]
            if len(current_users) > server_capacity[s]:
                sorted_users = sorted(current_users, key=lambda x: server_rank[s, x])
                server_matches[s] = sorted_users[:server_capacity[s]]
                for rej in sorted_users[server_capacity[s]:]:
                    free_users.append(rej)

    return server_matches

In [4]:
# === Greedy Assignment ===

def greedy_assignment(dist_matrix, server_capacity):
    """Assign each user to their closest available server."""
    num_users = dist_matrix.shape[0]
    assignments = []
    capacity_left = server_capacity.copy()
    for u in range(num_users):
        nearest_servers = np.argsort(dist_matrix[u])
        for s in nearest_servers:
            if capacity_left[s] > 0:
                assignments.append((u, s))
                capacity_left[s] -= 1
                break
    return assignments

In [8]:
# === Simulation Function ===

def simulate(env_id, num_slots=1000, distribution='exponential'):
    servers_pos, users_pos = generate_positions()
    dist_matrix, user_prefs, server_prefs = compute_preferences(users_pos, servers_pos, SERVER_SPEEDS)

    def effective_speed(u, s):
      return 1.0 / ((workloads[u] / SERVER_SPEEDS[s]) + dist_matrix[u, s] / workloads[u])

    # Task queues hold (remaining_work, original_work)
    server_tasks_stable = [[] for _ in range(NUM_SERVERS)]
    server_tasks_greedy = [[] for _ in range(NUM_SERVERS)]

    stable_times = []
    greedy_times = []

    for _ in range(num_slots):
        workloads = generate_workloads(NUM_USERS, distribution)

        # === STABLE Matching ===
        capacity_stable = []
        for s in range(NUM_SERVERS):
            updated = []
            for rem, orig, u in server_tasks_stable[s]:
                v = effective_speed(u, s)
                new_remain = rem - v * SLOT_DURATION
                if new_remain > 0:
                    updated.append((new_remain, orig, u))
                else:
                    stable_times.append(orig / v)
            server_tasks_stable[s] = updated
            capacity_stable.append(max(0, 2 - len(server_tasks_stable[s])))

        matching = many_to_one_stable_matching(user_prefs, server_prefs, np.array(capacity_stable))
        for s, users in matching.items():
            for u in users:
                server_tasks_stable[s].append((workloads[u], workloads[u], u))

        # === GREEDY Assignment ===
        capacity_greedy = []
        for s in range(NUM_SERVERS):
            updated = []
            for rem, orig, u in server_tasks_greedy[s]:
                v = effective_speed(u, s)
                new_remain = rem - v * SLOT_DURATION
                if new_remain > 0:
                    updated.append((new_remain, orig, u))
                else:
                    greedy_times.append(orig / v)
            server_tasks_greedy[s] = updated
            capacity_greedy.append(max(0, 2 - len(server_tasks_greedy[s])))

        assignments = greedy_assignment(dist_matrix, np.array(capacity_greedy))
        for u, s in assignments:
            server_tasks_greedy[s].append((workloads[u], workloads[u], u))

    return {
        "env_id": env_id,
        "stable_avg": np.mean(stable_times),
        "greedy_avg": np.mean(greedy_times),
        "stable_total": len(stable_times)/num_slots,
        "greedy_total": len(greedy_times)/num_slots,
    }


In [9]:
# === Run Single Environment Simulation ===

results = simulate(env_id=1)
print("Label:")
print(results)


Label:
{'env_id': 1, 'stable_avg': np.float64(5.9756690841754425), 'greedy_avg': np.float64(6.7261427848560915), 'stable_total': 5.274, 'greedy_total': 4.866}


In [10]:
# Run simulation for 10 different random environments and average the results
all_results = []

for env_id in range(1, 11):
    result = simulate(env_id=env_id, num_slots=1000, distribution='uniform')    # or exponential
    all_results.append(result)

# Convert to DataFrame and compute averages
results_df = pd.DataFrame(all_results)
avg_summary = pd.DataFrame({
    "Metric": ["Average Stable Completion Time", "Average Greedy Completion Time",
               "Average Stable Task Count", "Average Greedy Task Count"],
    "Value": [
        results_df["stable_avg"].mean(),
        results_df["greedy_avg"].mean(),
        results_df["stable_total"].mean(),
        results_df["greedy_total"].mean()
    ]
})

# Show full simulation results
print("=== Summary of 10 Environment Simulation ===")
print(results_df)

print("\n=== Averaged Metrics (uni dist) ===")
print(avg_summary)

=== Summary of 10 Environment Simulation ===
   env_id  stable_avg  greedy_avg  stable_total  greedy_total
0       1    6.336393    6.753135         5.297         5.075
1       2    6.253815    7.400747         5.356         4.811
2       3    6.466069    6.925256         5.182         4.949
3       4    5.612525    6.220241         5.535         5.339
4       5    5.222255    6.801890         5.837         4.980
5       6    6.325138    6.890526         5.283         5.011
6       7    5.378668    6.375227         5.708         5.156
7       8    5.655139    6.604931         5.552         5.102
8       9    7.073874    8.096665         4.939         4.470
9      10    5.887512    6.653411         5.442         5.147

=== Averaged Metrics (uni dist) ===
                           Metric     Value
0  Average Stable Completion Time  6.021139
1  Average Greedy Completion Time  6.872203
2       Average Stable Task Count  5.413100
3       Average Greedy Task Count  5.004000
